HomeWork_5

Петренко, М3104

Задание 1

Примечание: jupyter notebook для задания 1 и файл со списком ребер есть в репозитории по ссылке https://github.com/petmila/My_graphs_discrete_math

(a) Нарисовано с помощью сайта https://programforyou.ru/graph-redactor добавлением вершин и ребер вручную.

<img src="graph_europe.png"> 

Список вершин и их географических координат:

In [233]:
geolock = {"Armenia":{"height": 40,"width": 44},
           "Albania":{"height": 41,"width": 19},
           "Andorra":{"height": 42,"width": 1},
           "Austria":{"height": 48,"width": 16},
           "Belarus":{"height": 53,"width": 27},
           "Belgium":{"height": 50,"width": 4},
           "Bosnia_and_Herzegovina":{"height": 43,"width": 18},
           "Bulgaria":{"height": 42,"width": 23},
           "Croatia":{"height": 45,"width": 15},
           "Czech_Republic":{"height": 50,"width": 14},
           "Denmark":{"height": 55,"width": 12},
           "Estonia":{"height": 59,"width": 24},
           "Finland":{"height": 60,"width": 24},
           "France":{"height": 48,"width": 2},
           "Germany":{"height": 52,"width": 13},
           "Georgia":{"height": 41,"width": 44},
           "Greece":{"height": 37,"width": 23},
           "Hungary":{"height": 47,"width": 19},
           "Iceland":{"height": 64,"width": 21},
           "Ireland":{"height": 53,"width": 6},
           "Italy":{"height": 41,"width": 12},
           "Kosovo":{"height": 42,"width": 21},
           "Latvia":{"height": 56,"width": 24},
           "Liechtenstein":{"height": 47,"width": 9},
           "Lithuania":{"height": 54,"width": 25},
           "Luxembourg":{"height": 49,"width": 6},
           "Malta":{"height": 35,"width": 14},
           "Moldova":{"height": 47,"width": 28},
           "Monaco":{"height": 47,"width": 9},
           "Montenegro":{"height": 42,"width": 19},
           "Netherlands":{"height": 52,"width": 4},
           "North_Macedonia":{"height": 41,"width": 21},
           "Norway":{"height": 59,"width": 10},
           "Poland":{"height": 52,"width": 21},
           "Portugal":{"height": 38,"width": 9},
           "Romania":{"height": 47,"width": 19},
           "Russia":{"height": 55,"width": 37},
           "San_Marino":{"height": 43,"width": 12},
           "Serbia":{"height": 44,"width": 20},
           "Slovakia":{"height": 48,"width": 17},
           "Slovenia":{"height": 46,"width": 14},
           "Spain":{"height": 40,"width": 3},
           "Sweden":{"height": 59,"width": 18},
           "Switzerland":{"height": 46,"width": 7},
           "Turkey":{"height": 39,"width": 32},
           "Ukraine":{"height": 50,"width": 30},
           "United_Kingdom":{"height": 51,"width": 0},
           "Vatican_City":{"height": 41,"width": 12},
           }

In [234]:
import itertools
import matplotlib.pyplot as plt
import networkx as nx
import numpy.random as rnd
import haversine

class Europe:
    def __init__(self):
        # создание графа по списку ребер из файла(файл есть в репозитории)
        self.graph = nx.read_edgelist("graph.txt")
        # добавляю островные государства  
        self.graph.add_node('Malta')
        self.graph.add_node('Iceland')  
        self.graph.add_node('Cyprus')

        # мощности множеств вершин и ребер графа
        self.nodes_number = self.graph.number_of_nodes() 
        self.edges_number = self.graph.number_of_edges()

        # поиск наибольшей компоненты связности из списка всех компонент
        subgraphs = list(nx.connected_components(self.graph))
        self.connected_nodes_list = {}
        for subgraph in subgraphs:
            if len(subgraph) > len(self.connected_nodes_list):
                self.connected_nodes_list = subgraph
        self.largest_connected_component = nx.subgraph(self.graph, self.connected_nodes_list)

        # массив степеней всех вершин, чтобы найти минимальную и максимальную
        self.degree = nx.degree(self.largest_connected_component)  
        min_degree = self.nodes_number + 1
        max_degree = 0
        for node in self.degree:
            if node[1] < min_degree:
                min_degree = node[1]
            if node[1] > max_degree:
                max_degree = node[1]
        self.min_degree = min_degree
        self.max_degree = max_degree

        # раскраска - множество пар (вершина : номер цвета)
        self.v_coloring = nx.greedy_color(self.largest_connected_component) 
        # создание дуального графа и получение раскраски для ребер тем же способом
        self.dual = nx.line_graph(self.largest_connected_component) 
        self.e_coloring = nx.greedy_color(self.dual)

        cliques = list(nx.find_cliques(self.largest_connected_component))
        self.max_clique = cliques[0]
        for clique in cliques:
            if len(self.max_clique) < len(clique):
                self.max_clique = clique
            
        self.max_stable_set = nx.maximal_independent_set(self.largest_connected_component)

        # для невзвешенного графа дефолтны1 вес = 1, так что она посчитает просто количество ребер
        self.max_matching = nx.max_weight_matching(self.largest_connected_component)

        self.v_cover = nx.approximation.min_weighted_vertex_cover(self.largest_connected_component)
        self.e_cover = nx.min_edge_cover(self.largest_connected_component)

        self.blocks = list(nx.biconnected_components(self.graph))


    def number_of_nodes(self):
        print("Number of nodes:", self.nodes_number)

    def number_of_edges(self):
        print("Number of edges:", self.edges_number)

    def degree(self):
        print("Minimum degree:", self.min_degree)
        print("Maximum degree:", self.max_degree)

    def diameter(self):
        print("Diameter:", nx.diameter(self.largest_connected_component))

    def center(self):
        print("Center:", nx.center(self.largest_connected_component))

    def radius(self):
        print("Radius:", nx.radius(self.largest_connected_component))

    def girth(self):
        # поиск длины минимального цикла
        subgraphs = list(nx.minimum_cycle_basis(self.largest_connected_component))
        nodes_list = self.nodes_number * [0]
        for subgraph in subgraphs:
            if len(subgraph) < len(nodes_list):
                nodes_list = subgraph
        print("Girth:", len(nodes_list))
    
    def vertex_coloring(self):
        print("Minimum vertex coloring:", max(self.v_coloring.values()))

    def edge_coloring(self):
        print("Minimum edge coloring:", max(self.e_coloring.values()))

    def maximum_clique(self):
        print("Maximum clique:", self.max_clique)

    def maximum_stable_set(self):
        print("Maximum stable set:", self.max_stable_set)

    def maximum_matching(self):
        print("Maximum matching:", self.max_matching)

    def minimum_vertex_cover(self):
        print("Minimum vertex cover:", self.v_cover)
    
    def minimum_edge_cover(self):
        print("Minimum edge cover:", self.e_cover)

    def shortest_closed_walk_vertexes(self):
        # получаем список вершин
        self.tsp = (nx.algorithms.approximation.traveling_salesman_problem(self.largest_connected_component))
        print("Shortest closed walk, which visits every vetrex:\n")
        i = 0
        # вывод ребер, инцидентных вершинам из списка и самих вершин
        while i < (len(self.tsp) - 4):
            print(self.tsp[i], (self.tsp[i], self.tsp[i + 1]),
            self.tsp[i + 1], (self.tsp[i + 1], self.tsp[i + 2]),
            self.tsp[i + 2], (self.tsp[i + 2], self.tsp[i + 3]),
            self.tsp[i + 3], (self.tsp[i + 3], self.tsp[i + 4]))
            i += 4
        print(self.tsp[len(self.tsp) - 1])

    def shortest_closed_walk_edges(self):
        print("Shortest closed walk, which visits every edge:\n")
        # eulerize добавляет кратные ребра так, чтобы в не эйлеровом графе можно было построить ейлеров путь
        self.list_euler = list(nx.eulerian_circuit(nx.eulerize(self.largest_connected_component)))
        i = 0
        # вывод ребер из списка и вершин, им инцидентных
        print(self.list_euler[0][0])
        while i < (len(self.list_euler) - 4):
            print(self.list_euler[i], self.list_euler[i][1],
            self.list_euler[i + 1], self.list_euler[i + 1][1],
            self.list_euler[i + 2], self.list_euler[i + 2][1],
            self.list_euler[i + 3], self.list_euler[i + 3][1])
            i += 4

    def all_blocks(self):
        print("Biconnected components:")
        for item in self.blocks:
            print(item)

    def all_2_edge_connected_components(self):
        print("2-edge-connected components:")
        for item in list(nx.biconnected_component_edges(self.graph)):
            print(item)

    def make_weighted_graph(self):
        self.weighted_graph = nx.read_edgelist("graph.txt")  # создание графа по списку ребер из файла(файл есть в репозитории)
        self.weighted_graph.add_node('Malta')
        self.weighted_graph.add_node('Iceland')  # добавляю островные государства
        self.weighted_graph.add_node('Cyprus')

        nx.set_node_attributes(self.weighted_graph, geolock)
        edges_weight = {}
        # расчет весов как расстояний между географическими координатами городов
        for item in nx.edges(self.weighted_graph):
            edges_weight[item] = int(haversine.haversine((self.weighted_graph.nodes[item[0]]["height"], self.weighted_graph.nodes[item[0]]["width"]),
            (self.weighted_graph.nodes[item[1]]["height"],self.weighted_graph.nodes[item[1]]["width"])))
        nx.set_edge_attributes(self.weighted_graph , edges_weight, 'weight')

        self.largest_weighted_connected_component = nx.subgraph(self.weighted_graph, self.connected_nodes_list)
    
    def spanning_tree(self):
        self.minimum_spanning_tree = nx.minimum_spanning_tree(self.largest_weighted_connected_component)
        print("Minimum_spanning_tree:")
        return self.minimum_spanning_tree
    
    def prufer(self):
        print("Prufer code:", nx.to_prufer_sequence(nx.convert_node_labels_to_integers(self.minimum_spanning_tree)))

(b)

In [235]:
europe = Europe()
europe.number_of_edges()
europe.number_of_nodes()
europe.degree()
europe.diameter()
europe.center()
europe.radius()
europe.girth()


Number of edges: 92
Number of nodes: 49
Diameter: 8
Center: ['Poland']
Radius: 4
Girth: 3


Очевидно, что если в графе есть вершины, связанные с остальным графом только одним ребром, то:

𝜘(G) = 1 - минимальное число вершин, удалив которые, мы сделаем граф несвязным. 

𝜆(G) = 1 - минимальное число ребер, удалив которые, мы сделаем граф несвязным.

(c)

In [236]:
europe.vertex_coloring()

Minimum vertex coloring: 4


(d)


In [237]:
europe.edge_coloring()

Minimum edge coloring: 8


(e)

In [238]:
europe.maximum_clique()

Maximum clique: ['Belarus', 'Russia', 'Lithuania', 'Latvia']


(f)

In [239]:
europe.maximum_stable_set()

Maximum stable set: ['Russia', 'Czech_Republic', 'Luxembourg', 'Montenegro', 'Slovenia', 'Romania', 'Armenia', 'Portugal', 'Denmark', 'North_Macedonia', 'Netherlands', 'Andorra', 'Sweden', 'San_Marino', 'Liechtenstein', 'Vatican_City', 'Monaco']


(g)

In [240]:
europe.maximum_matching()

Maximum matching: {('Moldova', 'Romania'), ('Ukraine', 'Slovakia'), ('North_Macedonia', 'Albania'), ('Lithuania', 'Belarus'), ('Montenegro', 'Bosnia_and_Herzegovina'), ('Croatia', 'Hungary'), ('Bulgaria', 'Greece'), ('Poland', 'Czech_Republic'), ('Austria', 'Slovenia'), ('Estonia', 'Latvia'), ('Norway', 'Sweden'), ('Liechtenstein', 'Switzerland'), ('Turkey', 'Armenia'), ('Kosovo', 'Serbia'), ('Finland', 'Russia'), ('Monaco', 'France'), ('Vatican_City', 'Italy'), ('Portugal', 'Spain'), ('Netherlands', 'Belgium'), ('Denmark', 'Germany')}


(h)

In [241]:
europe.minimum_vertex_cover()

Minimum vertex cover: {'Belarus', 'Czech_Republic', 'Croatia', 'Hungary', 'Greece', 'Slovenia', 'Spain', 'North_Macedonia', 'Bulgaria', 'Austria', 'Ukraine', 'Serbia', 'Latvia', 'Germany', 'Romania', 'France', 'Switzerland', 'Finland', 'Georgia', 'Poland', 'Kosovo', 'Armenia', 'Belgium', 'Albania', 'Turkey', 'Slovakia', 'Bosnia_and_Herzegovina', 'Italy', 'Russia', 'Andorra', 'Sweden'}


(i)

In [242]:
europe.minimum_edge_cover()

Minimum edge cover: {('Moldova', 'Romania'), ('Belgium', 'Luxembourg'), ('Ukraine', 'Slovakia'), ('North_Macedonia', 'Albania'), ('Lithuania', 'Belarus'), ('Georgia', 'Armenia'), ('Andorra', 'Spain'), ('Montenegro', 'Bosnia_and_Herzegovina'), ('Luxembourg', 'Belgium'), ('Croatia', 'Hungary'), ('Armenia', 'Georgia'), ('Bulgaria', 'Greece'), ('Poland', 'Czech_Republic'), ('Spain', 'Andorra'), ('Austria', 'Slovenia'), ('Estonia', 'Latvia'), ('Norway', 'Sweden'), ('Liechtenstein', 'Switzerland'), ('Italy', 'San_Marino'), ('Turkey', 'Armenia'), ('Kosovo', 'Serbia'), ('San_Marino', 'Italy'), ('Finland', 'Russia'), ('Monaco', 'France'), ('Vatican_City', 'Italy'), ('Portugal', 'Spain'), ('Netherlands', 'Belgium'), ('Denmark', 'Germany')}


(j)

In [243]:
europe.shortest_closed_walk_vertexes()

Shortest closed walk, which visits every vetrex:

Armenia ('Armenia', 'Georgia') Georgia ('Georgia', 'Russia') Russia ('Russia', 'Norway') Norway ('Norway', 'Sweden')
Sweden ('Sweden', 'Finland') Finland ('Finland', 'Russia') Russia ('Russia', 'Estonia') Estonia ('Estonia', 'Latvia')
Latvia ('Latvia', 'Belarus') Belarus ('Belarus', 'Lithuania') Lithuania ('Lithuania', 'Poland') Poland ('Poland', 'Germany')
Germany ('Germany', 'Denmark') Denmark ('Denmark', 'Germany') Germany ('Germany', 'Netherlands') Netherlands ('Netherlands', 'Belgium')
Belgium ('Belgium', 'France') France ('France', 'Monaco') Monaco ('Monaco', 'France') France ('France', 'Luxembourg')
Luxembourg ('Luxembourg', 'Germany') Germany ('Germany', 'Czech_Republic') Czech_Republic ('Czech_Republic', 'Austria') Austria ('Austria', 'Slovenia')
Slovenia ('Slovenia', 'Croatia') Croatia ('Croatia', 'Serbia') Serbia ('Serbia', 'Montenegro') Montenegro ('Montenegro', 'Bosnia_and_Herzegovina')
Bosnia_and_Herzegovina ('Bosnia_and_H

(k)

In [244]:
europe.shortest_closed_walk_edges()

Shortest closed walk, which visits every edge:

Armenia
('Armenia', 'Georgia') Georgia ('Georgia', 'Russia') Russia ('Russia', 'Ukraine') Ukraine ('Ukraine', 'Romania') Romania
('Romania', 'Moldova') Moldova ('Moldova', 'Ukraine') Ukraine ('Ukraine', 'Hungary') Hungary ('Hungary', 'Slovenia') Slovenia
('Slovenia', 'Italy') Italy ('Italy', 'Vatican_City') Vatican_City ('Vatican_City', 'Italy') Italy ('Italy', 'San_Marino') San_Marino
('San_Marino', 'Italy') Italy ('Italy', 'Switzerland') Switzerland ('Switzerland', 'Liechtenstein') Liechtenstein ('Liechtenstein', 'Austria') Austria
('Austria', 'Switzerland') Switzerland ('Switzerland', 'Austria') Austria ('Austria', 'Slovenia') Slovenia ('Slovenia', 'Croatia') Croatia
('Croatia', 'Montenegro') Montenegro ('Montenegro', 'Bosnia_and_Herzegovina') Bosnia_and_Herzegovina ('Bosnia_and_Herzegovina', 'Croatia') Croatia ('Croatia', 'Hungary') Hungary
('Hungary', 'Ukraine') Ukraine ('Ukraine', 'Poland') Poland ('Poland', 'Lithuania') Lithuania (

(l)

<img src="graph_europe_block_cut_tree.png">

In [245]:
europe.all_blocks()

Biconnected components:
{'Spain', 'Portugal'}
{'Spain', 'France', 'Andorra'}
{'Italy', 'San_Marino'}
{'Vatican_City', 'Italy'}
{'France', 'Monaco'}
{'Norway', 'Russia', 'Finland', 'Sweden'}
{'Germany', 'Denmark'}
{'Belarus', 'Czech_Republic', 'Luxembourg', 'Croatia', 'Hungary', 'Lithuania', 'Greece', 'Slovenia', 'Netherlands', 'North_Macedonia', 'Bulgaria', 'Austria', 'Ukraine', 'Serbia', 'Latvia', 'Liechtenstein', 'Estonia', 'Germany', 'Montenegro', 'Romania', 'France', 'Switzerland', 'Poland', 'Kosovo', 'Belgium', 'Armenia', 'Albania', 'Turkey', 'Slovakia', 'Bosnia_and_Herzegovina', 'Italy', 'Russia', 'Moldova', 'Georgia'}
{'Ireland', 'United_Kingdom'}


Block-cut tree:

<img src="block_cut_tree.png">

(m)

In [246]:
europe.all_2_edge_connected_components()

2-edge-connected components:
[('Spain', 'Portugal')]
[('France', 'Andorra'), ('Andorra', 'Spain'), ('Spain', 'France')]
[('Italy', 'San_Marino')]
[('Italy', 'Vatican_City')]
[('France', 'Monaco')]
[('Russia', 'Finland'), ('Finland', 'Sweden'), ('Sweden', 'Norway'), ('Norway', 'Finland'), ('Norway', 'Russia')]
[('Germany', 'Denmark')]
[('Armenia', 'Turkey'), ('Turkey', 'Bulgaria'), ('Bulgaria', 'Greece'), ('Greece', 'Albania'), ('Albania', 'Serbia'), ('Serbia', 'Bosnia_and_Herzegovina'), ('Bosnia_and_Herzegovina', 'Croatia'), ('Croatia', 'Hungary'), ('Hungary', 'Austria'), ('Austria', 'Germany'), ('Germany', 'Belgium'), ('Belgium', 'France'), ('France', 'Luxembourg'), ('Luxembourg', 'Belgium'), ('Luxembourg', 'Germany'), ('France', 'Germany'), ('France', 'Switzerland'), ('Switzerland', 'Austria'), ('Switzerland', 'Germany'), ('Switzerland', 'Italy'), ('Italy', 'Austria'), ('Italy', 'France'), ('Italy', 'Slovenia'), ('Slovenia', 'Austria'), ('Slovenia', 'Croatia'), ('Slovenia', 'Hungary'

(n)

SPQR - дерево. Было нарисовано в sagemathcell тут:
https://sagecell.sagemath.org/?z=eJx9VE1r3DAQvS_sfxC52AGzNNBTIIc0tJvSpJQ0PQVjtPbEFitrXH3sV-l_70jxh3a37cn2zLw3b97IetXYMsNrWNSad41ZlKgUlFZshN0z0XaoLTPdT11YDTCfWXbDlr4y_ZXc6haU4Mk1e0menV7DPsmSJaCuKZhnya1c8ZB_SZYELoHS30GvKJQlX9DgBunlK3VoikdeQoWqBzpjNb0yz7wE3XLlqe8OUDbFE3RuJUXpySRu-DrQ3TtVc73vg0FWlny2XIbQVtgDaMlVRV8PgmgsKGNBKGo3nyUfQHLtjFf65IwJ4B9rzYXymr_hgOR2E3IPwjaO92oJXAvXevAnzVUYcxL94HbQrtDp2s8KtnnTYQISDXEU9Fncgz5AjRuh3vy808ht6DU69oiKVEOtMWCdpIGDSbH7g8-nrmbJE7ZcxYxh8qEPkUQW_qVlZGx-torreE3RVnrnCPDR2KCDxRb3flK6N46SR35FpPEKSffbavOphKDjnk72_Z8NDKc1VtWbmY9uUvLczqlq8I3KotGn43Nm_JGV4ywefqQ79rv_W3zNwHEu6fiA9OZ6w6fjmg1j5qf_ATvpn8enPDZ3InhEWdG8wbxxxmHuMFmkaJKeZ-xMPIvTQ69_GjoqGNqy2JhJwVBJ2Sg4csbh35fz2TOtm6638bZLLQX7K1CB3aJe7xg3TO3GcMttJ9FKsVp0e__m852081lFTGq3sFgM0CLcsKnvQrxUtHgVtdOQ0sOIA9ykV-8zdvXukrKErDTfFqYBKdMqo8U0heQrkObmWTvImMIKihIlamp0IUXd2JqY1UXPbRrcppd_AGj9-e0=&lang=sage&interacts=eJyLjgUAARUAuQ==

<img src="spqr_tree.png">

(o) 

In [247]:
europe.make_weighted_graph()
MST = europe.spanning_tree()
#for i in MST.nodes(): # можно вывести список ребер для каждой вершины, но он очень большой, поэтому я не стала его здесь приводить
#    print(MST.edges(i))
print(MST)

Minimum_spanning_tree:
Graph with 44 nodes and 43 edges


(p)
Минимальное остовное дерево было изображено вручную с помощью онлайн редактора графов по списку ребер для каждой вершины, полученному в предыдущем пункте.

<img src="spanning_tree.png"> 

Из рисунка видно, что центроидом является Austria.

(q)

In [248]:
europe.prufer()

Prufer code: [0, 1, 4, 7, 7, 29, 26, 26, 10, 7, 6, 31, 29, 30, 16, 15, 14, 12, 13, 11, 24, 25, 37, 36, 21, 20, 10, 17, 17, 16, 11, 22, 20, 25, 23, 14, 11, 19, 18, 10, 8, 9]


<img src="proofs_5_HW/1.png">
<img src="proofs_5_HW/2.png">
<img src="proofs_5_HW/3.png">
<img src="proofs_5_HW/4.png">
<img src="proofs_5_HW/5.png">